# Objective

This notebook primarily loads, cleans and save a cleaned version of the provided dataset.

# Prep notebook

In [1]:
import pandas as pd

from booking_forecaster.configs import params

# Load data

In [2]:
df_l0 = pd.read_csv(params.DATA_PATH / "hotel_bookings.csv", delimiter=",")

In [3]:
df_l0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   hotel_name            150000 non-null  object 
 1   season                150000 non-null  object 
 2   arrival_date          150000 non-null  object 
 3   departure_date        150000 non-null  object 
 4   guest_type            150000 non-null  object 
 5   customer_type         150000 non-null  object 
 6   company               150000 non-null  float64
 7   adr                   150000 non-null  float64
 8   num_guests            150000 non-null  float64
 9   adults                150000 non-null  int64  
 10  children              149995 non-null  float64
 11  babies                150000 non-null  int64  
 12  meal                  150000 non-null  object 
 13  source_country        150000 non-null  object 
 14  market_segment        150000 non-null  object 
 15  

In [4]:
df_l0.head()

,hotel_name,season,arrival_date,departure_date,guest_type,customer_type,company,adr,num_guests,adults,children,babies,meal,source_country,market_segment,distribution_channel,assigned_room_type,is_canceled,booking_date
0,Algarve Retreat,Spring,2015-07-07,2015-08-02,Group,Transient-Party,40.0,86.313825,2.0,2,0.0,0,BB,PRT,Online TA,Direct,A,0,2014-07-09
1,Algarve Retreat,Spring,2015-07-16,2015-08-02,Family,Transient,40.0,59.115150,1.0,1,0.0,0,HB,PRT,Groups,TA/TO,A,1,2014-07-19
2,Algarve Retreat,Spring,2015-07-06,2015-07-22,Family,Transient-Party,40.0,81.292354,2.0,2,0.0,0,BB,PRT,Online TA,TA/TO,A,0,2014-07-27
3,Algarve Retreat,Summer,2015-07-16,2015-08-08,Couple,Transient,40.0,110.208364,1.0,1,0.0,0,SC,FRA,Online TA,TA/TO,A,0,2014-07-27
4,Algarve Retreat,Autumn,2015-07-18,2015-08-13,Couple,Transient,40.0,107.166776,3.0,3,0.0,0,BB,FRA,Direct,Corporate,A,0,2014-07-30


# Data prep

In [5]:
df_l1 = df_l0.copy(deep=True)

From domain knowledge and inferring from the field name we could cast the `*_date` fields to `pd.Timestamp`

In [6]:
date_cols = [
    "arrival_date",
    "booking_date",
    "departure_date"
]

for col in date_cols:
    df_l1[col] = pd.to_datetime(df_l1[col], format="%Y-%m-%d")

In [7]:
df_l1[date_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   arrival_date    150000 non-null  datetime64[ns]
 1   booking_date    150000 non-null  datetime64[ns]
 2   departure_date  150000 non-null  datetime64[ns]
dtypes: datetime64[ns](3)
memory usage: 3.4 MB


Let's try to do some basic inspection on the dataset

From domain knowledge, we assume that the field `company` is some kind of an id and therefore we cast it to type `category` for now.

In [8]:
df_l1['company'] = df_l1['company'].astype(int).astype('category')

In [9]:
# count number of unique elements in the non-numeric cols
df_l1.select_dtypes(['object', 'category']).nunique()

hotel_name                5
season                    4
guest_type                4
customer_type             4
company                 331
meal                      5
source_country          170
market_segment            8
distribution_channel      5
assigned_room_type       11
dtype: int64

Here on, it is safe to assume that all the above types are some kind of categories so we cast them accordingly. 

In [10]:
object_cols = df_l1.select_dtypes('object').columns

In [11]:
df_l1[object_cols] = df_l1[object_cols].astype('category')

Again from domain knowledge and inferring from the field name `is_canceled` could be cast to `boolean`. But before that let's actually count the number of uniques to verify this assumption

In [12]:
df_l1['is_canceled'].nunique()

2

In [13]:
df_l1['is_canceled'] = df_l1['is_canceled'].astype(bool)

In [14]:
# check the records for which `children` have null values
df_l1.loc[df_l1['children'].isna()]

,hotel_name,season,arrival_date,departure_date,guest_type,customer_type,company,adr,num_guests,adults,children,babies,meal,source_country,market_segment,distribution_channel,assigned_room_type,is_canceled,booking_date
26343,Braga City Hotel,Autumn,2015-12-01,2015-12-13,Couple,Transient,40,68.070924,1.960426,2,NaN,0,BB,PRT,Offline TA/TO,TA/TO,A,False,2015-01-22
106208,Lisbon City Hotel,Summer,2015-08-03,2015-08-04,Family,Transient-Party,40,13.135240,1.962701,2,NaN,0,BB,PRT,Undefined,Undefined,B,True,2015-08-01
106209,Lisbon City Hotel,Summer,2015-08-05,2015-08-07,Family,Transient-Party,40,13.806969,1.962701,2,NaN,0,BB,PRT,Direct,Undefined,B,True,2015-08-04
137729,Porto City Hotel,Winter,2018-01-01,2018-01-13,Couple,Transient,40,75.116746,1.958684,2,NaN,0,SC,PRT,Offline TA/TO,TA/TO,A,False,2017-06-29
139516,Porto City Hotel,Summer,2015-08-05,2015-08-07,Family,Transient-Party,40,19.770300,1.958684,3,NaN,0,BB,PRT,Undefined,Undefined,B,True,2015-08-04


Since everything else in other columns look fine - we will fill the null values for these 5 records with 0 for now

In [15]:
df_l1['children'].fillna(0, inplace=True)

Again, just from domain knowledge and inferring from the field names we can make the following statements:
1. The field `num_guests`, `adults`, `children` and `babies` should be of type `int` and must be always >= 0.
2. The field `num_guests` must be >= 1.
3. Also for every row $num\_guests = adults + children + babies$

Let's verify each of the above hypotheses.

Assertion 1

In [16]:
int_cols = ["num_guests", "adults", "children", "babies"]

In [17]:
def assert_int(df, cols):
    try: 
        left = df[cols].sum().sum()
        right = df[cols].astype(int).sum().sum()
        assert left == right, f"Absolute diff is {abs(left - right)}" 
    except AssertionError as ex:
        print(f"Assertion failed for columns - {cols} !. Error message - {ex}")

In [18]:
assert_int(df_l1, int_cols)

Assertion failed for columns - ['num_guests', 'adults', 'children', 'babies'] !. Error message - Absolute diff is 4.803196590859443


So we may need to have a closer look for each column to understand if the problem is across all the fields or any one in particular

In [19]:
for col in int_cols:
    assert_int(df_l1, col)

Assertion failed for columns - num_guests !. Error message - Absolute diff is 4.803196590801235


So now we want to have a closer look in the records where the field `num_guests` has non-integer values

In [20]:
df_l1.loc[df_l1['num_guests'] != df_l1['num_guests'].astype(int)]

,hotel_name,season,arrival_date,departure_date,guest_type,customer_type,company,adr,num_guests,adults,children,babies,meal,source_country,market_segment,distribution_channel,assigned_room_type,is_canceled,booking_date
26343,Braga City Hotel,Autumn,2015-12-01,2015-12-13,Couple,Transient,40,68.070924,1.960426,2,0.0,0,BB,PRT,Offline TA/TO,TA/TO,A,False,2015-01-22
106208,Lisbon City Hotel,Summer,2015-08-03,2015-08-04,Family,Transient-Party,40,13.135240,1.962701,2,0.0,0,BB,PRT,Undefined,Undefined,B,True,2015-08-01
106209,Lisbon City Hotel,Summer,2015-08-05,2015-08-07,Family,Transient-Party,40,13.806969,1.962701,2,0.0,0,BB,PRT,Direct,Undefined,B,True,2015-08-04
137729,Porto City Hotel,Winter,2018-01-01,2018-01-13,Couple,Transient,40,75.116746,1.958684,2,0.0,0,SC,PRT,Offline TA/TO,TA/TO,A,False,2017-06-29
139516,Porto City Hotel,Summer,2015-08-05,2015-08-07,Family,Transient-Party,40,19.770300,1.958684,3,0.0,0,BB,PRT,Undefined,Undefined,B,True,2015-08-04


So here on we have the following options:
1. We throw these records away, OR
2. We round them up and keep them.

For now, I would go with option 2

In [21]:
df_l1['num_guests'] = round(df_l1['num_guests']).astype(int)

Now let's check if the interger cols all are >= 0 or not

In [22]:
assert (df_l1[int_cols] < 0).sum().sum() == 0, "One or more columns have negative values"

Assertion 2

In [23]:
assert (df_l1['num_guests'] < 1).sum() == 0, "The column `num_guests` have values < 1"

Assertion 3

In [24]:
sum_series = df_l1["adults"] + df_l1["children"] + df_l1["babies"]

In [25]:
anomalous_num_guests = df_l1["num_guests"] != sum_series

In [26]:
try: 
    assert anomalous_num_guests.sum() == 0, "For some records the summation assumption doesn't hold true"
except AssertionError as ex:
    print(f"Assertion failed. Error message: {ex}")

Assertion failed. Error message: For some records the summation assumption doesn't hold true


In [27]:
df_l1.loc[anomalous_num_guests]

,hotel_name,season,arrival_date,departure_date,guest_type,customer_type,company,adr,num_guests,adults,children,babies,meal,source_country,market_segment,distribution_channel,assigned_room_type,is_canceled,booking_date
139516,Porto City Hotel,Summer,2015-08-05,2015-08-07,Family,Transient-Party,40,19.7703,2,3,0.0,0,BB,PRT,Undefined,Undefined,B,True,2015-08-04


Since the above is just one record we could throw it away

In [28]:
df_l1 = df_l1[~anomalous_num_guests]

We could safely cast the `children` column to `int` now.

In [29]:
df_l1['children'] = df_l1['children'].astype(int)

Let's check the range of the values in the integer columns

In [30]:
print(df_l1[int_cols].min())
print(df_l1[int_cols].max())

num_guests    1
adults        1
children      0
babies        0
dtype: int64
num_guests    57
adults        55
children       3
babies        10
dtype: int64


Optionally we could further down cast these columns to `uint8` datatype -  this would save some memory footprint but since we are not actually dealing with out-of-core data here we skip doing it for now.

Let's have a look into another important field - `adr` (average daily rate).

In [31]:
df_l1['adr'].min(), df_l1['adr'].max()

(-28.972876193327767, 462.6388013072548)

Let's have a closer look in the number of records for which `adr` is negative

In [32]:
negative_adr = df_l1['adr'] < 0

In [33]:
df_l1.loc[negative_adr]

,hotel_name,season,arrival_date,departure_date,guest_type,customer_type,company,adr,num_guests,adults,children,babies,meal,source_country,market_segment,distribution_channel,assigned_room_type,is_canceled,booking_date
799,Algarve Retreat,Autumn,2015-12-05,2015-12-31,Couple,Transient,40,-15.383247,2,2,0,0,BB,PRT,Complementary,Direct,A,False,2015-02-24
1682,Algarve Retreat,Autumn,2015-07-19,2015-08-05,Couple,Transient,40,-1.866959,1,1,0,0,BB,BEL,Complementary,TA/TO,A,False,2015-06-09
2163,Algarve Retreat,Summer,2016-02-05,2016-02-23,Couple,Transient,40,-14.868839,1,1,0,0,BB,PRT,Complementary,TA/TO,A,False,2015-07-21
3814,Algarve Retreat,Summer,2016-01-08,2016-01-11,Couple,Transient,40,-13.431002,2,2,0,0,BB,FRA,Complementary,TA/TO,A,False,2015-11-09
4477,Algarve Retreat,Spring,2016-06-10,2016-06-19,Couple,Transient,40,-1.378447,2,2,0,0,SC,GBR,Complementary,Direct,A,False,2016-01-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121350,Porto City Hotel,Summer,2016-01-27,2016-02-12,Single,Transient,40,-4.047084,1,1,0,0,BB,CN,Complementary,TA/TO,A,True,2015-08-21
121643,Porto City Hotel,Summer,2016-04-28,2016-05-13,Single,Transient,40,-1.712676,1,1,0,0,BB,IRL,Complementary,TA/TO,A,True,2015-09-04
133531,Porto City Hotel,Summer,2017-03-14,2017-03-26,Couple,Group,40,-0.388213,2,2,0,0,BB,PRT,Complementary,TA/TO,A,True,2016-12-18
138135,Porto City Hotel,Autumn,2018-01-19,2018-02-05,Couple,Transient-Party,40,-4.247148,1,1,0,0,HB,IRL,Complementary,TA/TO,A,True,2017-08-05


The values in the `adr` column essentially denote prices (according to page - 4 of the PDF of task instructions) - negative prices usually mean a refund. Which usually happen in cases of cancellations. Let's verify if that's true.

In [34]:
df_l1.loc[negative_adr]['is_canceled'].value_counts()

is_canceled
False    48
True     19
Name: count, dtype: int64

Which means that our assumption is wrong. So in order to proceed from here for the sake of data sanity we could throw them away but first let's see if we would lose some valuable information by doing this. 

This is especially important if we have some categories that only appear in these records.

So let's first see if that's the case.

In [35]:
cat_cols = df_l1.select_dtypes('category').columns

In [36]:
for col in cat_cols:
    cats_in_positive_adr = set(df_l1.loc[~negative_adr][col])
    cats_in_negative_adr = set(df_l1.loc[negative_adr][col])
    if not cats_in_negative_adr.issubset(cats_in_positive_adr):
        print(f"These categories in {col} are only present in the records with negative ADR {cats_in_negative_adr - cats_in_positive_adr}")

So thankfully that's not the case, let's check if we have some numeric values which are only present in these records

In [37]:
for col in int_cols:
    minimum_in_negative_adr = df_l1.loc[negative_adr][col].min()
    minimum_in_positive_adr = df_l1.loc[~negative_adr][col].min()
    maximum_in_negative_adr = df_l1.loc[negative_adr][col].max()
    maximum_in_positive_adr = df_l1.loc[~negative_adr][col].max()

    if not minimum_in_positive_adr <= minimum_in_negative_adr <= maximum_in_negative_adr <= maximum_in_positive_adr:
        print(f"The column {col} has values in the records with negative ADR that are not seen in the ones with positive ADR")

For numeric fields its not the case, let's check the same for the date columns

In [38]:
for col in date_cols:
    minimum_in_negative_adr = df_l1.loc[negative_adr][col].min()
    minimum_in_positive_adr = df_l1.loc[~negative_adr][col].min()
    maximum_in_negative_adr = df_l1.loc[negative_adr][col].max()
    maximum_in_positive_adr = df_l1.loc[~negative_adr][col].max()

    if not minimum_in_positive_adr <= minimum_in_negative_adr <= maximum_in_negative_adr <= maximum_in_positive_adr:
        print(f"The column {col} has dates in the records with negative ADR that are not seen in the ones with positive ADR")

So looking at all the fields we could conclude that not considering these 67 records should not largely change the picture in our latter analyses

In [39]:
df_l1 = df_l1.loc[~negative_adr]

Next, let's check for some anomalies in the date columns. As a basic check we could say that for all records the following should hold true:
1. $booking\_date <= arrival\_date <= departure\_date$

In [40]:
anomalous_dates = ~((df_l1['booking_date'] <= df_l1['arrival_date']) &\
        (df_l1['arrival_date'] <= df_l1['departure_date']))

In [41]:
try: 
    assert anomalous_dates.sum() == 0
except AssertionError as ex:
    print(f"Some records have anomalous dates")

Finally we need to check and drop any duplicate records if we have. P.S. in this dataset we do not have any field that uniquely identifies a booking (e.g. something like booking-id). Therefore, we need to be careful when aggressively dropping duplicate records.

In [42]:
after = len(df_l1.drop_duplicates())
before = len(df_l1)
perc_dropped = abs(after-before)/before
if perc_dropped > 0.05:
    raise Exception(f"More than 5% ({before-after}) of records are getting dropped. Please check !")
else:
    df_l1 = df_l1.drop_duplicates()

In [43]:
# finalize preliminary data prep
df_l1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 149899 entries, 0 to 149999
Data columns (total 19 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   hotel_name            149899 non-null  category      
 1   season                149899 non-null  category      
 2   arrival_date          149899 non-null  datetime64[ns]
 3   departure_date        149899 non-null  datetime64[ns]
 4   guest_type            149899 non-null  category      
 5   customer_type         149899 non-null  category      
 6   company               149899 non-null  category      
 7   adr                   149899 non-null  float64       
 8   num_guests            149899 non-null  int64         
 9   adults                149899 non-null  int64         
 10  children              149899 non-null  int64         
 11  babies                149899 non-null  int64         
 12  meal                  149899 non-null  category      
 13  sour

# Save data

In [44]:
df_l1.reset_index(drop=True).to_parquet(params.DATA_PATH / "hotel_bookings_l1.parquet")